In [6]:
import numpy as np
from rdp import rdp

# Example data inspection and usage
data_array = np.array([
    [0, 0, 0, 0],
    [1, 1, 1, 1],
    [2, 2, 2, 2],
    [3, 3, 3, 3]
])

# Check shape of data_array before passing to rdp
print("Shape of data_array:", data_array.shape)

# Call rdp function
try:
    epsilon = 1
    simplified_data = rdp(data_array, epsilon=epsilon)
    print("Simplified data:", simplified_data)
except ValueError as e:
    print("Error:", e)

Shape of data_array: (4, 4)
Error: incompatible dimensions for cross product
(dimension must be 2 or 3)
